# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb8486e1e80>
├── 'a' --> tensor([[-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805]])
└── 'x' --> <FastTreeValue 0x7fb8486e1e50>
    └── 'c' --> tensor([[ 0.0311, -1.4602, -0.5836, -0.2576],
                        [-1.4393,  1.6647,  0.7051, -0.0036],
                        [ 0.9901,  0.2347,  1.9765,  1.1955]])

In [4]:
t.a

tensor([[-0.8355, -1.0703,  1.5445],
        [ 1.8138, -0.7282, -0.7805]])

In [5]:
%timeit t.a

59.7 ns ± 0.039 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb8486e1e80>
├── 'a' --> tensor([[-0.0139,  0.5999,  0.8151],
│                   [-1.5821,  1.0812, -0.2161]])
└── 'x' --> <FastTreeValue 0x7fb8486e1e50>
    └── 'c' --> tensor([[ 0.0311, -1.4602, -0.5836, -0.2576],
                        [-1.4393,  1.6647,  0.7051, -0.0036],
                        [ 0.9901,  0.2347,  1.9765,  1.1955]])

In [7]:
%timeit t.a = new_value

61.2 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8355, -1.0703,  1.5445],
               [ 1.8138, -0.7282, -0.7805]]),
    x: Batch(
           c: tensor([[ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955]]),
       ),
)

In [10]:
b.a

tensor([[-0.8355, -1.0703,  1.5445],
        [ 1.8138, -0.7282, -0.7805]])

In [11]:
%timeit b.a

56 ns ± 0.0508 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1068,  0.5276,  0.3291],
               [ 0.2599, -0.7940, -0.0113]]),
    x: Batch(
           c: tensor([[ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 19.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 349 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 418 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb79c3a7130>
├── 'a' --> tensor([[[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]],
│           
│                   [[-0.8355, -1.0703,  1.5445],
│                    [ 1.8138, -0.7282, -0.7805]]])
└── 'x' --> <FastTreeValue 0x7fb79c3a7e20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 74.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb79c47d130>
├── 'a' --> tensor([[-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805],
│                   [-0.8355, -1.0703,  1.5445],
│                   [ 1.8138, -0.7282, -0.7805]])
└── 'x' --> <FastTreeValue 0x7fb8486da760>
    └── 'c' --> tensor([[ 0.0311, -1.4602, -0.5836, -0.2576],
                        [-1.4393,  1.6647,  0.7051, -0.0036],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 41.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 65.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0311, -1.4602, -0.5836, -0.2576],
                       [-1.4393,  1.6647,  0.7051, -0.0036],
                       [ 0.9901,  0.2347,  1.9765,  1.1955]],
              
                      [[ 0.0311, -1.4602, -0.5836, -0.2576],
                       [-1.4393,  1.6647,  0.7051, -0.0036],
                       [ 0.9901,  0.2347,  1.9765,  1.1955]],
              
                      [[ 0.0311, -1.4602, -0.5836, -0.2576],
                       [-1.4393,  1.6647,  0.7051, -0.0036],
                       [ 0.9901,  0.2347,  1.9765,  1.1955]],
              
                      [[ 0.0311, -1.4602, -0.5836, -0.2576],
                       [-1.4393,  1.6647,  0.7051, -0.0036],
                       [ 0.9901,  0.2347,  1.9765,  1.1955]],
              
                      [[ 0.0311, -1.4602, -0.5836, -0.2576],
                       [-1.4393,  1.6647,  0.7051, -0.0036],
                       [ 0.9901,  0.2347,  1.9765,  1.1955]],

In [26]:
%timeit Batch.stack(batches)

76.8 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955],
                      [ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955],
                      [ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955],
                      [ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955],
                      [ 0.0311, -1.4602, -0.5836, -0.2576],
                      [-1.4393,  1.6647,  0.7051, -0.0036],
                      [ 0.9901,  0.2347,  1.9765,  1.1955],
                      [ 0.0311, -1.4602, -0.5836, -0.2576],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
